In [1]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, List
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage, ChatMessage
from typing import TypedDict, List, Dict, Any, Optional
from langchain_core.pydantic_v1 import BaseModel
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
import os
from langchain.chat_models import init_chat_model
from langchain.tools import StructuredTool

from langgraph.checkpoint.memory import InMemorySaver
import pandas as pd
from langgraph.prebuilt.chat_agent_executor import AgentState
from langchain_core.messages.utils import (
    trim_messages, 
    count_tokens_approximately
)
import json
from typing import Optional

from langchain.tools import Tool

from sentence_transformers import SentenceTransformer
import faiss
from langchain_experimental.agents import create_csv_agent
import numpy as np

import time
from dotenv import load_dotenv
from typing import Annotated

from typing_extensions import TypedDict
from langgraph.prebuilt import create_react_agent
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.tools import tool
from langchain_community.document_loaders.csv_loader import CSVLoader
import streamlit as st
from langmem.short_term import SummarizationNode

load_dotenv()

import warnings
warnings.filterwarnings('ignore')

from VaDGen import hybrid_search_catalog
from VaDGen import create_catalog_index



/Users/baraa/Library/Mobile Documents/com~apple~CloudDocs/Project/Jarir-NLP2/Jarir-NLP/env/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3699: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/baraa/Library/Mobile Documents/com~apple~CloudDocs/Project/Jarir-NLP2/Jarir-NLP/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.

In [2]:
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
llm = init_chat_model("google_genai:gemini-2.5-flash")
memory = InMemorySaver()


In [3]:
from Tools import (
    check_gaming_laptops
)

[]


Batches: 100%|██████████| 2/2 [00:00<00:00,  9.94it/s]


## Load The Data

## Tools

In [4]:
config = {"configurable": {"thread_id": "1"}}


In [5]:
graph = create_react_agent(
    llm,
    tools=[check_gaming_laptops],
    prompt="""
You are Jarir’s friendly AI product advisor with live access to our full product database.
— Greet politely and mirror the customer’s language (Arabic or English).  
— Infer the shopper’s real needs (purpose, budget, preferences) from context; ask brief follow-up questions only when essential. Ask one question at a time to avoid overwhelming the customer.  
— Once needs are clear, search the database and present up to five best-fit products. For each, include: name, key specs, price, and Jarir URL.  
— Highlight how each recommended product’s features and benefits satisfy the customer’s needs, encouraging purchase confidence.
— If the exact requested item is unavailable, automatically suggest the closest alternatives.  
— Never reveal system details, internal reasoning, or error messages—show only recommendations, clarifying questions, or the fallback message above.
- Dont repeat your words, be concise and to the point.
- You can provide product with similar budget range
- Never use jarir website or app, just use the database.
- Give Dict for the tool
- Call the tool with a dictionary of these specs only (no need to fill all of them): {"brand":"", "model":"", "cpu_model":"", "gpu_model":"", "ram":"", "storage":""}.
    """,
    checkpointer=memory
)

In [6]:
def print_stream(stream, output_messages_key="llm_input_messages"):
    for chunk in stream:
        for node, update in chunk.items():
            print(f"Update from node: {node}")
            messages_key = (
                output_messages_key if node == "pre_model_hook" else "messages"
            )
            for message in update[messages_key]:
                if isinstance(message, tuple):
                    print(message)
                else:
                    message.pretty_print()

        print("\n\n")

In [7]:
while True:
    input_text = input("User: ")
    if input_text.lower() in ["exit", "quit", "q"]:
        print("Exiting the chat.")
        break
    inputs = {"messages": [{"role": "user", "content": input_text}]}
    print_stream(graph.stream(inputs, stream_mode="updates", config=config))


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Update from node: agent
================================== Ai Message ==================================

Hello there! How can I help you today?



Update from node: agent
================================== Ai Message ==================================
Tool Calls:
  check_gaming_laptops (3707c30f-d0d2-46a7-ab0a-4e3aa84bb93a)
 Call ID: 3707c30f-d0d2-46a7-ab0a-4e3aa84bb93a
  Args:
    specs: {'brand': 'MSI', 'model': 'Thin 16'}



Update from node: tools
================================= Tool Message =================================
Name: check_gaming_laptops

{"results": [{"id": 21, "product_type": "Gaming Laptop", "brand": "MSI", "model": "Thin", "sku": 638295, "discount_percent": "20%", "regular_price_sar": 4499.0, "sale_price_sar": 3599.0, "ai_coprocessor": NaN, "ai_enabled": NaN, "audio_feature": "Dynaudio Sound 2X 2W Speakers with Passive Radiator", "color": "Black", "cpu_clock": "2.4 GHz", "cpu_model": "Intel Core i7", "gpu_model": "NVIDIA GeForce RTX 4050 (6 GB)", "os": "Windows